Below is setup for Drake

In [0]:
try:
    import pydrake
    import underactuated
except ImportError:
    !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
    from jupyter_setup import setup_underactuated
    setup_underactuated()

# Setup matplotlib.
from IPython import get_ipython
if get_ipython() is not None: get_ipython().run_line_magic("matplotlib", "inline")

In [0]:
# python libraries
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import sys 
import platform

# pydrake imports
from pydrake.all import (MultibodyPlant, Parser, DiagramBuilder, Simulator,
                         PlanarSceneGraphVisualizer, SceneGraph, TrajectorySource,
                         SnoptSolver, MultibodyPositionToGeometryPose, PiecewisePolynomial,
                         MathematicalProgram, JacobianWrtVariable, eq, LinearQuadraticRegulator,
                         Linearize, AddMultibodyPlantSceneGraph)
# underactuated imports
from underactuated import FindResource

In [52]:
#clears file and downloads urdf to have it be updated
get_ipython().system(u"rm -rf /opt/hopper")
if platform.system() == "Darwin":
  get_ipython().system(u"if [ ! -d '/opt/hopper' ]; then git clone https://github.com/Olliebear/underactuated_final_project.git /opt/hopper; fi")
elif platform.linux_distribution() == ("Ubuntu", "18.04", "bionic"):
  get_ipython().system(u"if [ ! -d '/opt/hopper' ]; then git clone https://github.com/Olliebear/underactuated_final_project.git /opt/hopper; fi" )
else:
  assert False, "Unsupported platform"
sys.path.append("/opt/hopper")

Cloning into '/opt/hopper'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 25 (delta 11), reused 19 (delta 5), pack-reused 0
Unpacking objects: 100% (25/25), done.


##Define the problem's parameters:
x0, y0 are coordinates of the foot of the 2d hopper.

th1 (theta1) is the angle between the foot and the upright, th2 is angle between bode and upright.

z0 is the distance from the hip pin joint to the foot. 

uT is torque input at pinjoint connecting leg and body. uX is length actuation on the spring in the leg. 

###Other parameters:
Spring length K_length= z0 - uX.

m1,I1,r1 = mass, inertia and radius from toe to C.O.M. for the leg

m2,I2,r2 = mass, inertia and radius from hip to C.O.M. for the body


#Equations of motion:
Many different modes of dynamics. It makes sense to use Mixed-Integer MPC methods to control. Let's define our modes and the equations that go with them.
##While in the air:


##While on the ground:
For now, assume infinite friction. Can add friction cone constraint later, or add more modes

###Define Some System Constants
Order of states: x0, y0, th1, z0, th2

In [0]:
x_initial=[0, 3, 0, 0, 0, 0, 0, 0, 0, 0]


# weight matrices for the lqr controller
Q = np.eye(10)
R = np.eye(4)

In [101]:
builder=DiagramBuilder()
hopper, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
Parser(hopper).AddModelFromFile('/opt/hopper/hopper.urdf')
hopper.Finalize()
context=hopper.CreateDefaultContext()
context.SetTime(0)
context.SetContinuousState(x_initial)




<pydrake.systems.framework.OutputPort_[float] object at 0x7f2f36205830>


RuntimeError: ignored